In [ ]:
import json

with open('/home/giacomoschiavo/finetuning-BirdNET/utils/dataset/true_segments_test.json') as f:
    true_segments = json.load(f)

selected_segments = true_segments['20190601_030000.WAV'] 